In [ ]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import make_column_transformer

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
df = pd.read_excel(url)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['Minute'] = df['InvoiceDate'].dt.minute
df['Invoice_str'] = df['Invoice'].astype(str)
df['CustomerID_str'] = df['Customer ID'].astype(str)
df['StockCode_str'] = df['StockCode'].astype(str)
df.dropna(how= 'any')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Month,Year,Day,Hour,Minute,Invoice_str,CustomerID_str,StockCode_str
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,12,2009,1,7,45,489434,13085.0,85048
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,12,2009,1,7,45,489434,13085.0,79323P
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,12,2009,1,7,45,489434,13085.0,79323W
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,12,2009,1,7,45,489434,13085.0,22041
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,12,2009,1,7,45,489434,13085.0,21232


In [ ]:
df['Quarter'] = df['InvoiceDate'].dt.quarter
df['TotalAmount'] = df['Quantity'] * df['Price']

In [ ]:
# Feature Engineering Puchase Variables For Each Unique Customer

MonthlyPurchaseQuantity = df.groupby(['Customer ID', 'Month'])['Quantity'].sum().reset_index()
df = pd.merge(df, MonthlyPurchaseQuantity, on= ['Customer ID', 'Month'], suffixes= ['', '_perMonth'])

MonthlyPurchaseAmount = df.groupby(['Customer ID', 'Month'])['TotalAmount'].sum().reset_index()
df = pd.merge(df, MonthlyPurchaseAmount, on= ['Customer ID', 'Month'], suffixes= ['', '_perMonth'])

YearlyPurchaseQuantity = df.groupby(['Customer ID', 'Year'])['Quantity'].sum().reset_index()
df  = pd.merge(df, YearlyPurchaseQuantity, on= ['Customer ID', 'Year'], suffixes= ['', '_perYear'])

YearlyPurchaseAmount = df.groupby(['Customer ID', 'Year'])['TotalAmount'].sum().reset_index()
df = pd.merge(df, YearlyPurchaseAmount, on= ['Customer ID', 'Year'], suffixes= ['', '_perYear'])

In [ ]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Month', 'Year', 'Day', 'Hour',
       'Minute', 'Invoice_str', 'CustomerID_str', 'StockCode_str', 'Quarter',
       'TotalAmount', 'Quantity_perMonth', 'TotalAmount_perMonth',
       'Quantity_perYear', 'TotalAmount_perYear'],
      dtype='object')

############################ Classification ####################### Can we determine if a customer will be a repeat customer based on purchase history using classification models? Use oldest 80% of the sales data in training a classification model, and use most recent 20% as a validation set for evaluating accuracy of our predictions. Use feature engineering for whether or not was repeat customer.

In [ ]:
# Define a repeat customer
df['RepeatCustomer'] = (df.groupby('CustomerID')['InvoiceNo'].transform('nunique') > 1).astype(int)

# Feature Engineering
df['Return'] = (df['Quantity'] < 0).map({True: 'Yes', False: 'No'})

# Define predictors and target
x_predictors = df[['StockCode_str', 'CustomerID_str', 'Country', 'Price', 'Month', 'Day', 'Hour', 'Minute']]
y_result = df['Return']

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_predictors, y_result, train_size=0.8, shuffle=False)

# Preprocessing
ct = make_column_transformer(
    (StandardScaler(), ['Price']),
    (OneHotEncoder(handle_unknown='ignore'), ['StockCode_str', 'CustomerID_str', 'Country']),
    remainder='passthrough'
)

# Classification model
class_model_return = make_pipeline(
    ct,
    KNeighborsClassifier(n_neighbors=5)
)

# Train the model
class_model_return.fit(x_train, y_train)

# Scoring for the classification model
# Cross-validation scores for training set
train_cv_scores = cross_val_score(class_model_return, x_train, y_train, cv=4, scoring="accuracy")
train_accuracy = train_cv_scores.mean()

# Convert 'Yes'/'No' labels to boolean values for F1 score calculation
train_returned = (y_train == 'Yes')

# Calculate F1 score using cross-validation for training set
train_f1 = cross_val_score(class_model_return, x_train, train_returned, cv=10, scoring="f1").mean()

# Display results
print("Training Accuracy:", train_accuracy)
print("Training F1 Score:", train_f1)

KeyError: 'CustomerID'

In [ ]:
# Negative quantity in the data set reflects an item being returned
Returns = df[df['Quantity'] < 0]
Returns.head(2)

In [ ]:
df['Return'] = df['Quantity'] < 0
df['Return'] = df['Return'].map({
    True: 'Yes',
    False: 'No'
})
df.head(2)

In [ ]:
x_predictors = df[['StockCode_str', 'CustomerID_str', 'Country', 'Price', 'Month', 'Day', 'Hour', 'Minute']] # Did not include year as this does not provide much additional value
y_result = df['Return']

x_train, x_val, y_train, y_val = train_test_split(x_predictors, y_result, train_size= 0.1) # Using only 10% of data set for training due to limited computation resources and large data set size

ct = make_column_transformer(
    (StandardScaler(), ['Price']),
    (OneHotEncoder(handle_unknown='ignore'), ['StockCode_str', 'CustomerID_str', 'Country']),
    remainder= 'passthrough'
)
ct

class_model_return = make_pipeline(
    ct,
    KNeighborsClassifier(n_neighbors= 5)
)

class_model_return.fit(x_train, y_train)

In [ ]:
# Scoring for classification model - Note: Validation sets are not tested due to limited computational resources though implementation is shown below

train_cv_scores = cross_val_score(class_model_return, x_train, y_train,
                            cv=4, scoring="accuracy")         # CV of 4 is used due to large data set size
train_accuracy = train_cv_scores.mean()

#val_cv_scores = cross_val_score(class_model_return, x_val, y_val,
#                            cv=4, scoring="accuracy")
#val_accuracy = cv_scores.mean()

train_returned = (y_train == 'Yes')
#val_returned = (y_val == 'Yes')


train_f1 = cross_val_score(class_model_return, x_train, train_returned,
                            cv=10, scoring="f1").mean()

#val_f1 = cross_val_score(class_model_return, x_val, val_returned,
 #                           cv=10, scoring="f1").mean()


In [ ]:
print("Test Training Accuracy: ", train_accuracy)
print("Test Training f1: ", train_f1)

In [ ]:
"""
# Optimize Overall Accuracy
grid_search = GridSearchCV(
    m1,
    param_grid={"kneighborsclassifier__n_neighbors": range(1, 10)},
    scoring="accuracy",
    cv=4
)

grid_search.fit(X_train, y_train)

accuracy = cross_val_score(
    grid_search.best_estimator_,
    X_train, survived_titanic,
    scoring="accuracy",
    cv=10).mean()

print(grid_search.best_params_)
print("Accuracy: ", accuracy)
